# Torch
## Check GPU¶

In [1]:
import torch
import numpy as np
import sys
sys.path.append('..')

from torchlib.utils import list_device,set_device

# S1: check GPU
#list_device()

# S2: default parameters
set_device(1)
np.set_printoptions(precision = 2)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

Using Device 1 : TITAN Xp


the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


# Set Arguments

In [2]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()

'''Training Parameters'''
parser.add_argument('--batch_size', type=int, default=5000, help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200, help='number of epochs')
parser.add_argument('--grad_clip', type=float, default=5., help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.007, help='learning rate')
parser.add_argument('--learning_rate_clip', type=float, default=0.0000001, help='learning rate clip')
parser.add_argument('--decay_rate', type=float, default=.8, help='decay rate for rmsprop')
parser.add_argument('--weight_decay', type=float, default=.0001, help='decay rate for rmsprop')
parser.add_argument('--batch_norm_decay', type=float, default=.999, help='decay rate for rmsprop')
parser.add_argument('--keep_prob', type=float, default=1.0, help='dropout keep probability')
parser.add_argument('--lamda_weights', type=float, default=.01, help='lamda weight')
parser.add_argument('--data_argumentation', type=bool, default=True, help='whether do data argument')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data nomalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')
parser.add_argument('--output_dim', default=3, type=int, help='output dimention.')
parser.add_argument('--feat_dim', default=128, type=int, help='feature dimention.')

'''Configure'''
parser.add_argument('--network', type=str, default='vggnet_localization')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/gp_net_torch', help='rnn, gru, or lstm')

parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/train_dense_old'])

parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

parser.add_argument('--seed', default=1337, type=int)
parser.add_argument('--save_every', type=int, default=2000, help='save frequency')
parser.add_argument('--display', type=int, default=10, help='display frequency')
parser.add_argument('--tensorboard', type=bool, default=True, help='open tensorboard')
parser.add_argument('--cuda_device', type=int, default=1, help='cuda device')


sys.argv = ['']
args = parser.parse_args()

# Load Dataset

In [3]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
from PIL import Image
import numpy as np
import random

import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.cnn_auxiliary import normalize, denormalize_navie, denormalize, get_relative_pose, translational_rotational_loss
from torchlib.utils import LocalizationDataset, display_loss, data2tensorboard
import time

path = args.train_dataset[0]
x_dense = torch.load(os.path.join(path,'x.pt'))
y_dense = torch.load(os.path.join(path,'y.pt'))
dataset = TensorDataset(x_dense, y_dense)

[args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=True, pin_memory=True)

Load norm and std: /notebooks/global_localization/norm_mean_std.pt


# Define Model

In [4]:
from torch.cuda.amp import autocast, GradScaler
from torchlib.GPs import Backbone, NN, BaseModule

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        #context_feat = self.global_context(input_data)
        context_feat = input_data
        output,feature_t, feature_r = self.global_regressor(context_feat)
        return output, feature_t, feature_r

class GP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points, output_dim=3):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([output_dim])
        )
        variational_strategy = gpytorch.variational.MultitaskVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ), num_tasks=output_dim
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ZeroMean()
        #self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([3]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([3])),
            batch_shape=torch.Size([3]))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        inducing_points = torch.zeros(3, 1000, 128)
        self.backbone = Backbone()
        self.nn = NN()
        self.gp = GP(inducing_points)
        self.likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)
        
    def forward(self, input_data):
        #dense_feat = self.backbone(input_data)
        dense_feat = input_data
        output, feature_t, feature_r = self.nn(dense_feat)
        rot_pred = torch.split(output, [3, 4], dim=1)[1] # 4-dimention 
        trans_pred = self.gp(feature_t)
        return trans_pred, rot_pred

class PosePredictor(BaseModule):
    def __init__(self, norm_mean, norm_std, args,
                 is_training=True, mixed_precision=True,
                 regressor_context_rate = [0.0,0.0], train_rot = False):
        super().__init__(norm_mean, norm_std, args)
        self.model = Model().to(self.device)
        self.train_rot = train_rot
        self.mixed_precision = mixed_precision
        if self.mixed_precision:
            self.scaler = GradScaler()

        self.disable_requires_grad(self.model.backbone)
        
        if is_training:
            self.optimizer = optim.Adam(self._optimize(regressor_context_rate))
            self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                             lr_lambda=lambda epoch: args.decay_rate**epoch)
            num_data = min(len(dataloader)*args.batch_size,len(dataset))
            self.mll = gpytorch.mlls.PredictiveLogLikelihood(self.model.likelihood, \
                                                         self.model.gp, num_data = num_data)
        else:
            self.disable_requires_grad(self.model)
            
    def _optimize(self,regressor_context_rate):
        # GP
        optimizer = [
                {'params': self.model.gp.parameters(), \
                 'lr': args.learning_rate,'weight_decay':args.weight_decay},
                {'params': self.model.likelihood.parameters(), \
                 'lr': args.learning_rate,'weight_decay':args.weight_decay}]
            
        # NN
        if regressor_context_rate[0]!=0:
            optimizer += [{'params': self.model.nn.global_regressor.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[0],'weight_decay':args.weight_decay}]
            print('Regressor learn rate:',regressor_context_rate[0])
        else:
            self.disable_requires_grad(self.model.nn.global_regressor)
                
        if regressor_context_rate[1]!=0:
            optimizer += [{'params': self.model.nn.global_context.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[1],'weight_decay':args.weight_decay}]
            print('Context learn rate:',regressor_context_rate[1])
            self.train_rot = True
        else:
            self.disable_requires_grad(self.model.nn.global_context)
            
        
        if not self.train_rot and regressor_context_rate[1]==0.0:
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc1_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc2_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc3_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.logits_r)
                
        return optimizer
    
    def train(self,x, y):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        if args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
            
        # Step 2: training
        assert trainer.model.training == True
        if self.mixed_precision:
            with autocast():
                single_loss = self._loss(x, y)
            self.scaler.scale(single_loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            single_loss = self._loss(x, y)
            single_loss.backward()
            self.optimizer.step()
            
        batch_time = time.time() - start
        
        return float(single_loss), batch_time
    
    def _loss(self,x, y):
        # target
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        # predict
        trans_pred, rot_pred = self.model(x)
        
        # trans loss
        trans_loss = -1.*self.mll(trans_pred, trans_target)
        # rot loss
        rot_loss = 1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_target),dim=1)))
        
        total_loss = trans_loss + args.lamda_weights * rot_loss      
        
        return total_loss
    
    def eval_forward(self,x,y,num_sample = 100,output_denormalize = True):
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        
        # Step 2: forward
        assert trainer.model.training == False
        if self.mixed_precision:
            with autocast():
                trans_prediction, rot_prediction = self.model(x)
            self.scaler.scale(trans_prediction)
            self.scaler.scale(rot_prediction)
        else:
            trans_prediction, rot_prediction = self.model(x)
            
        trans_prediction, trans_mean, trans_var = self._eval_gp(trans_prediction)
        
        if args.is_normalization and output_denormalize:
            trans_prediction = denormalize_navie(trans_prediction, self.norm_mean, self.norm_std)
            trans_mean = denormalize_navie(trans_mean, self.norm_mean, self.norm_std)
            trans_var = trans_var.mul(self.norm_std)
        
        samples = self._sample(trans_mean, trans_var, num_sample)
            
        # Step 3: split output
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        
        return trans_prediction, rot_prediction, trans_target, rot_target, samples
    
    def _sample(self, mean, var, num_sample = 100):
        dist = torch.distributions.Normal(mean, var)
        samples = dist.sample([num_sample])
        return samples
    
    def _eval_gp(self, trans_pred):
        c_mean, c_var = trans_pred.mean, trans_pred.variance
        y_mean, y_var = self.model.likelihood(trans_pred).mean, self.model.likelihood(trans_pred).variance
        
        return y_mean, c_mean, c_var

In [5]:
trainer = PosePredictor(args.norm_mean,args.norm_std,args,mixed_precision=False,
                        regressor_context_rate = [0.05,0.], train_rot = False)
#trainer.load_model('pretrained.pth',strict = False)
#trainer.load_model('model-fast.pth')
trainer.load_model('model-fast-zero-mean.pth')
#trainer.show_require_grad()

Regressor learn rate: 0.05
Successfully loaded model to TITAN Xp.


In [6]:
'''
state_dict = torch.load(os.path.join(args.model_dir, 'pretrained.pth'))
state_dict = trainer._from_old_model(state_dict)
for key in list(state_dict):
    if 'gp' in key or 'likelihood' in key:
        state_dict.pop(key)
trainer.model.load_state_dict(state_dict, strict=False)
'''

"\nstate_dict = torch.load(os.path.join(args.model_dir, 'pretrained.pth'))\nstate_dict = trainer._from_old_model(state_dict)\nfor key in list(state_dict):\n    if 'gp' in key or 'likelihood' in key:\n        state_dict.pop(key)\ntrainer.model.load_state_dict(state_dict, strict=False)\n"

# Training

In [7]:
if args.tensorboard:
    import os
    os.system('rm -rf runs/gp')
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/gp')
import time
time.sleep(3)

In [10]:
trainer.model.train()
for e in range(args.num_epochs):
    minibatch_iter = tqdm(dataloader)
    train_loss = 0.
    for b, data in enumerate(minibatch_iter):
        x,y = data

        loss, batch_time = trainer.train(x,y)
        
        train_loss += loss
        ave_loss = train_loss/(b+1)
        step = e*len(dataloader)+(b+1)
        # display data
        minibatch_iter.set_postfix(ave = ave_loss, loss=loss,lr=trainer.scheduler.get_last_lr()[0])
        # tensorboard
        trainer.data2tensorboard(writer,'training loss',{'item loss':loss,'batch loss':ave_loss},step)
        # save model
        trainer.save_model_step(e,step)
        # step scheduler
        trainer.schedule_step(step,25)

 51%|█████     | 26/51 [00:29<00:28,  1.12s/it, ave=-9.18, loss=-9.17, lr=4.13e-5]


KeyboardInterrupt: 

In [11]:
trainer.save_model('model-fast-zero-mean.pth')

Saving model to model-fast-zero-mean.pth


In [12]:
trainer.model.eval()
trans_prediction, rot_prediction, trans_target, rot_target, samples = trainer.eval_forward(x,y)

In [13]:
trans_prediction

tensor([[  66.3477,  242.2132,   -2.5149],
        [-282.7759,  441.0593,  -12.2314],
        [  77.3453,  225.7924,   -2.1146],
        ...,
        [-303.7842,  503.8400,  -12.1227],
        [  47.1907,  181.5075,   -2.0454],
        [-195.7864,  666.3806,  -12.7685]], device='cuda:1',
       grad_fn=<AddBackward0>)

In [14]:
trans_target

tensor([[  66.2164,  242.2283,   -2.5141],
        [-282.5126,  441.7702,  -12.2235],
        [  76.8958,  225.4434,   -2.0860],
        ...,
        [-302.6446,  503.5599,  -12.1453],
        [  47.0254,  181.1609,   -2.0274],
        [-195.4930,  666.0799,  -12.7509]], device='cuda:1')

In [9]:
trainer.save_model('model-fast.pth')

Saving model to model-fast.pth
